> Please go through Giba's post and kernel  to underrstand what this leak is all about
> https://www.kaggle.com/titericz/the-property-by-giba (kernel)
> https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61329 (post)
> 
> Also, go through this Jiazhen's kernel which finds more columns to exploit leak
> https://www.kaggle.com/johnfarrell/giba-s-property-extended-result
> 
> I just exploit data property in brute force way and then fill in remaining by row non zero means! This should bring everyone on level-playing field.
> 
> **Let the competition begin! :D**

### Just some small modifications from [original baseline](https://www.kaggle.com/tezdhar/breaking-lb-fresh-start)~
- The leak rows are calculated separately on train/test set
- Calculated the leaky values, correctness, for each lag
- Hope this can help to do some *lag_selection*

### Update leak process codes to Dmitry Frumkin's *fast* [version](https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2)
- The result of Dmitry's original function and result of Hasan's function seem slightly different
- Modified to make the output consistent with Hasan's function (Seems better score)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# print(os.listdir("../input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

# Any results you write to the current directory are saved as output.

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [3]:
test["target"] = train["target"].mean()

# Determine Extra Features to use

In [4]:
rows = [1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245, 1383, 2071, 3492, 378, 2971, 2366, 4414, 2790, 3979, 193, 1189, 3516, 810, 4443, 3697, 235, 1382, 4384, 3418, 4396, 921, 3176, 650]

def element_wise_comparison(i1, i2, thresh=3):
    i1 = np.array(i1)
    i2 = np.array(i2)
    dif = i1 - i2
    count = np.count_nonzero(dif)
    if count <= thresh:
        return True
    else:
        return False

def solve_ties_in_nb_of_fillers(df, counts, stop_thresh=2):
    args = np.argsort(counts)[::-1]
    stop = np.squeeze(np.where(counts[args] == len(df.index)-stop_thresh))
    try:
        stop = int(stop)
    except:
          return [None]
    u, c = np.unique(counts[args[stop:]], return_counts=True)
    conflicts = u[np.where(c > 1)]
    for c in conflicts:
        idx = np.squeeze(np.where(counts == c))
        ref_id = np.squeeze(np.where(counts == c+1))
        ref = df.iloc[:-1, int(ref_id)]
        dif1 = np.count_nonzero(df.iloc[1:, idx[0]].values - ref.values)
        dif2 = np.count_nonzero(df.iloc[1:, idx[1]].values - ref.values)
        loc0  = np.where(args == idx[0])
        loc1  = np.where(args == idx[1])
        tmp0 = args[loc0]
        tmp1 = args[loc1]

        if dif2 < dif1:
            args[np.minimum(loc0, loc1)] = tmp1
            args[np.maximum(loc0, loc1)] = tmp0
        else:
            args[np.minimum(loc0, loc1)] = tmp0
            args[np.maximum(loc0, loc1)] = tmp1           

    return args[stop:]
    
        
def find_filler_value(df, idx):
    u, c = np.unique(df.iloc[0,:].values, return_counts=True)
    arg = np.argsort(c)
    return u[arg][-idx], c[arg][-idx]

def find_col_order_by_filler(df, filler, stop_thresh=2, nb=40):
    counts = []
    for c in df.columns:
        v = df[c].values - filler
        counts.append(np.count_nonzero(v))
    counts = np.array(counts)
    args = solve_ties_in_nb_of_fillers(df, counts, stop_thresh)
    if not all(args):
        return [None]
    return df.iloc[:, args]

def find_leading_col(raw_df, sorted_df, thresh=3):
    curr_lead = sorted_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'}).iloc[0, 0][1:]
    cols_to_match = raw_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'})
    for t in range(thresh+1):
        for c in cols_to_match.index:
                if element_wise_comparison(cols_to_match.loc[c, 'key'][:-1], curr_lead, thresh=t):
                    print('Leading col found: {}'.format(c))
                    return pd.concat([raw_df[c], sorted_df], axis=1)
        
    print('No lead found')
    return sorted_df
    
def find_n_cols_groups(df, n_groups=40, stop_thresh=2, col_count_tresh=30):
    col_groups = []
    for i in range(n_groups):
        filler, count = find_filler_value(df, i)
        if count <10:
            continue
        df_view = find_col_order_by_filler(df, filler)
        if not all(df_view):
            extra_cols = find_col_order_by_filler_for_ambiguous_cols(df, filler)
            col_groups.extend(extra_cols)
        else:
            for _ in range(stop_thresh + 1):
                df_view = find_leading_col(df, df_view)
            col_groups.append(list(df_view.columns))
            
    # Remove cols groups with less cols than tresh
    col_groups = [x for x in col_groups if len(x) >= col_count_tresh]
    return col_groups

def find_bottom_col(raw_df, sorted_df, thresh=4):
    curr_bot = sorted_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'}).iloc[-1, 0][:-1]
#     print(curr_bot)
    cols_to_match = raw_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'})
    for t in range(thresh):
        for c in cols_to_match.index:
    #         print(cols_to_match.loc[c, 'key'][1:])
            if element_wise_comparison(cols_to_match.loc[c, 'key'][1:], curr_bot, thresh=t):
                print('Bottom col found: {}'.format(c))
                return pd.concat([sorted_df, raw_df.pop(c)], axis=1), True
    
    print('No bottom found')
    return sorted_df, False

def remove_unsutable_candidates(candidates_pool, ref,  thresh=1):
    unique = ref.iloc[:, 0].unique()
    for c in candidates_pool.columns:
        tmp =  0
        for u in unique:
            if u not in candidates_pool[c].values:
                tmp +=1 
            if tmp > thresh:
                candidates_pool = candidates_pool.drop(c, axis=1)
                break
    return candidates_pool
    
def find_col_order_by_filler_for_ambiguous_cols(df, filler, n_cols=40):
    counts = []
    for c in df.columns:
        v = df[c].values - filler
        counts.append(np.count_nonzero(v))
    counts = np.array(counts)
    args = np.argsort(counts)[::-1]
    stop = np.squeeze(np.where(counts[args] == len(df.index)-2))
    extra_cols = []
#     return df.iloc[:, args[min(stop):]]
    for j in range(len(stop)):
        sorted_cols = df.iloc[:, args[stop[j]]].to_frame()
        candidates_pool = df.iloc[:, args[min(stop):]].copy()
#         candidates_pool = remove_unsutable_candidates(candidates_pool, sorted_cols, 2)

        for _ in range(n_cols-3):
            sorted_cols, state = find_bottom_col(candidates_pool, sorted_cols)
            if not np.count_nonzero(sorted_cols.iloc[:, -1].values - filler) or not state:
                break

        for i in range(2):
            sorted_cols = find_leading_col(df, sorted_cols, thresh=3)
        extra_cols.append(list(sorted_cols.columns))
    return extra_cols

# Analysis

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
tmp = train.loc[rows, :].drop('ID', axis=1)
x = find_col_order_by_filler_for_ambiguous_cols(tmp, 0)
# extra_feats = [list(x.columns) for x in find_col_order_by_filler_for_ambiguous_cols(tmp, 0)]
# [len(x) for x in extra_feats]

No bottom found
Leading col found: 2ef57c650
Leading col found: 4fa02e1a8
Bottom col found: 64cabb6e7
Bottom col found: 93521d470
Bottom col found: 46c525541
Bottom col found: cef9ab060
Bottom col found: 375c6080e
No bottom found
Leading col found: 85ef8a837
Leading col found: f99a09543
Bottom col found: 861076e21
Bottom col found: 9de83dc23
No bottom found
Leading col found: 9de83dc23
Leading col found: 935ca66a9
No bottom found
Leading col found: 8b5c0fb4e
Leading col found: 67102168f
Bottom col found: 1702b5bf0
Bottom col found: 324921c7b
Bottom col found: 62e59a501
No bottom found
Leading col found: b43a7cfd5
Leading col found: d6bb78916
Bottom col found: 2288333b4
Bottom col found: f39074b55
Bottom col found: a8b721722
Bottom col found: 13ee58af1
Bottom col found: fb387ea33
Bottom col found: 4da206d28
Bottom col found: ea4046b8d
No bottom found
Leading col found: 5d3b81ef8
Leading col found: e222309b0
Bottom col found: 9ca0eee11
Bottom col found: 4933f2e67
Bottom col found: fe33df

Leading col found: 5b318b659
Bottom col found: 37c10d610
Bottom col found: 7e3e026f8
Bottom col found: 5d9f43278
No bottom found
Leading col found: 7e3e026f8
Leading col found: 5d9f43278
Bottom col found: c3c3f66ff
No bottom found
Leading col found: 7f91dc936
Leading col found: 57c4c03f6
Bottom col found: da22ed2b8
Bottom col found: 89cebceab
Bottom col found: f171b61af
Bottom col found: 3a07a8939
Bottom col found: 129fe0263
Bottom col found: e5b2d137a
No bottom found
Leading col found: 79cc300c7
Leading col found: 20551fa5b
Bottom col found: 66ace2992
Bottom col found: f74e8f13d
Bottom col found: 5c6487af1
Bottom col found: 963a49cdc
No bottom found
Leading col found: 241f0f867
Leading col found: 2ec5b290f
No bottom found
Leading col found: d0f65188c
Leading col found: adf357c9b
No bottom found
Leading col found: 58d9f565a
Leading col found: b77bc4dac
No bottom found
Leading col found: 324921c7b
Leading col found: 1702b5bf0
Bottom col found: f39074b55
Bottom col found: a8b721722
Botto

Leading col found: 05b0f3e9a
Leading col found: ca96df1db
Bottom col found: 3aab2691c
Bottom col found: 3ecc09859
Bottom col found: cb5329038
Bottom col found: 8de6fcbf1
Bottom col found: b96c4256a
Bottom col found: a78f85d49
No bottom found
Leading col found: 8de6fcbf1
Leading col found: a78f85d49
Bottom col found: 956d228b9
Bottom col found: 949ed0965
Bottom col found: a4511cb0b
Bottom col found: b64425521
No bottom found
Leading col found: 8618bc1fd
Leading col found: 64c6eb1cb
No bottom found
Leading col found: a3fb07bfd
Leading col found: 467bee277
No bottom found
Leading col found: deabe0f4c
Leading col found: ea5ed6ff7


In [12]:
x = y
x

[['4fa02e1a8', '2ef57c650', '9aeec78c5'],
 ['f99a09543',
  '85ef8a837',
  'a31ba11e6',
  '64cabb6e7',
  '93521d470',
  '46c525541',
  'cef9ab060',
  '375c6080e'],
 ['935ca66a9', '9de83dc23', '861076e21', '861076e21', '9de83dc23'],
 ['67102168f', '8b5c0fb4e', '14a22ab1a'],
 ['d6bb78916',
  'b43a7cfd5',
  '58232a6fb',
  '1702b5bf0',
  '324921c7b',
  '62e59a501'],
 ['e222309b0',
  '5d3b81ef8',
  '1184df5c2',
  '2288333b4',
  'f39074b55',
  'a8b721722',
  '13ee58af1',
  'fb387ea33',
  '4da206d28',
  'ea4046b8d'],
 ['cc507de6c',
  'e0bb9cf0b',
  '80b14398e',
  '9ca0eee11',
  '4933f2e67',
  'fe33df1c4'],
 ['3d71c02f0', '05c9b6799', '26df61cc3'],
 ['d4cc42c3d',
  '1351bf96e',
  '1e8801477',
  'b7d59d3b5',
  'a459b5f7d',
  '580f5ff06',
  '39b3c553a',
  '1eec37deb'],
 ['f74e8f13d', '5c6487af1', '963a49cdc'],
 ['b850c3e18',
  '212efda42',
  '9e7c6b515',
  '2d065b147',
  '49ca7ff2e',
  '37c85a274',
  'ea5ed6ff7',
  'deabe0f4c',
  'bae4f747c'],
 ['504c22218',
  '56c679323',
  '002d634dc',
  '19388

In [15]:
# x = []
# for i in y:
#     x.extend(i)
# x = list(set(x))
# len(x)
tmp = train.loc[rows, x]
tmp.to_csv('sort_this.csv')

In [6]:
tmp = train.loc[rows, :].drop('ID', axis=1)
u, c = np.unique(tmp.iloc[0,:].values, return_counts=True)
arg = np.argsort(c)[::-1]
u[arg], c[arg]
# u[np.where(c == 36)]

(array([        0.  ,   1032000.  ,   1500000.  ,   1674000.  ,
          4000000.  ,   3000000.  ,   2992315.78,   2865694.28,
          2831000.  ,   1775333.34,  45138000.  ,   2160000.  ,
          2002166.66,   3160000.  ,   2000000.  ,   1964666.66,
          2400000.  ,   1612000.  ,   3255483.88,   1563411.76,
          1470000.  ,   1166666.66,   1033000.  ,    812666.66,
           803538.46,    726000.  ,    662000.  ,    400000.  ,
           210000.  ,     80000.  ,   3249000.  ,  70448000.  ,
          2202500.  ,   6568857.14,  40770000.  ,   3822750.  ,
         14650000.  ,  11028000.  ,   4434140.  ,   7822000.  ,
          7440000.  ,   4677000.  ,   5300000.  ,   5422000.  ,
          5600000.  ,   5921285.72,   6902000.  ,    790000.  ,
           200000.  ,   1830000.  ,  34714000.  ,    604000.  ,
           253333.34,    273000.  ,    182000.  ,    278000.  ,
           174000.  ,    346000.  ,    360000.  ,    144000.  ,
           380000.  ,    120000.  ,    1

In [ ]:
tmp = test.loc[rows, :].drop('ID', axis=1)
u, c = np.unique(tmp.iloc[0,:].values, return_counts=True)
arg = np.argsort(c)[::-1]
u[arg], c[arg]

# Calculate extra features

In [5]:
tmp = train.loc[rows, :].drop('ID', axis=1)
extra_feats = find_n_cols_groups(tmp, 85)
# extra_feats
len(extra_feats)

No bottom found
Leading col found: 2ef57c650
Leading col found: 4fa02e1a8
Bottom col found: 64cabb6e7
Bottom col found: 93521d470
Bottom col found: 46c525541
Bottom col found: cef9ab060
Bottom col found: 375c6080e
Bottom col found: a31ba11e6
Bottom col found: 85ef8a837
Bottom col found: f99a09543
Bottom col found: c83fc48f2
Bottom col found: cde3e280a
Bottom col found: d6af4ee1a
Bottom col found: 337b3e53b
Bottom col found: c46028c0f
No bottom found
Leading col found: 85ef8a837
Leading col found: f99a09543
Bottom col found: 861076e21
Bottom col found: 9de83dc23
Bottom col found: 935ca66a9
Bottom col found: a09a238d0
Bottom col found: f97d9431e
Bottom col found: bb0ce54e9
Bottom col found: 11e12dbe8
No bottom found
Leading col found: 9de83dc23
Leading col found: 935ca66a9
Bottom col found: 14a22ab1a
No bottom found
Leading col found: 8b5c0fb4e
Leading col found: 67102168f
Bottom col found: 1702b5bf0
Bottom col found: 324921c7b
Bottom col found: 62e59a501
No bottom found
Leading col foun

Bottom col found: 25e2bcb45
Bottom col found: a2e5adf89
Bottom col found: 043e4971a
Bottom col found: 3e3ea106e
Bottom col found: b0868a049
Bottom col found: 4baa9ff99
Bottom col found: 4b1d463d7
Bottom col found: b219e3635
Bottom col found: 6b6cd5719
Bottom col found: 63509764f
Bottom col found: e3a41c199
Bottom col found: 1529e64f6
Bottom col found: 2af3668d1
Leading col found: e1d6a5347
Leading col found: e88913510
Bottom col found: 324921c7b
Bottom col found: 62e59a501
No bottom found
Leading col found: 58232a6fb
Leading col found: b43a7cfd5
Bottom col found: a4511cb0b
Bottom col found: b64425521
Bottom col found: 949ed0965
No bottom found
Leading col found: 956d228b9
Leading col found: 6b795a2bc
Bottom col found: 5755fe831
Bottom col found: 91ace30bd
Bottom col found: 84067cfe0
Bottom col found: 15e4e8ee5
Bottom col found: d01cc5805
Bottom col found: 2b6f74f09
Bottom col found: b6403de0b
Bottom col found: 8761d9bb0
Bottom col found: 50a6c6789
Bottom col found: fe3fe2667
No bottom 

Bottom col found: 5cbf1b3ea
Bottom col found: 879e1f51a
Bottom col found: aaed60a3d
Bottom col found: 8909b68e1
Bottom col found: 82ade3db6
Bottom col found: 0a03426de
Bottom col found: 14d2c6d95
Bottom col found: 0a4e510ee
Bottom col found: 9c6bf2983
Bottom col found: 2862eec4f
Bottom col found: 406a11b5a
Bottom col found: e2e6f1d23
Bottom col found: 1a67a2101
Bottom col found: 08d17e384
Bottom col found: 3268914c7
Bottom col found: 0738fe5ff
Bottom col found: 11d9e8383
Bottom col found: 43dc0f90c
Bottom col found: 49f11e712
Bottom col found: eda856f5f
Bottom col found: e662f1672
Bottom col found: ee9e70298
Bottom col found: fb8395d97
Bottom col found: 1a9b9e2fe
Bottom col found: e3a41c199
Bottom col found: 1529e64f6
Leading col found: 760ef38f1
Leading col found: 2322dbbbb
Bottom col found: 4d2671746
Bottom col found: 822e49b95
Bottom col found: 3c8a3ced0
Bottom col found: 83635fb67
Bottom col found: 1857fbccf
Bottom col found: c4972742d
No bottom found
Leading col found: d7d314edc
L

Bottom col found: c270cb02b
Bottom col found: 03b8b1f2d
Bottom col found: 5a798adc1
Bottom col found: 71659d45d
Bottom col found: 621833d9b
Bottom col found: 6786ea46d
Bottom col found: 1110cf9ea
Bottom col found: df3cf10bd
Bottom col found: df5f5272a
Bottom col found: e8f43b3ab
Bottom col found: 26cc05472
Bottom col found: a939664aa
Bottom col found: 17b81a716
Bottom col found: 90e069c70
Bottom col found: de5e5b296
Bottom col found: 3f8854df3
Bottom col found: 1d81e197a
Bottom col found: e8b599c85
Bottom col found: 5271beb00
Bottom col found: 59cafde1f
Bottom col found: 346c04a64
Bottom col found: fb2ab1513
Bottom col found: 7106396d7
Bottom col found: de0aaf6f4
Bottom col found: 95bbebdd4
Bottom col found: b6636b458
Bottom col found: 74fa547c6
Bottom col found: aca96197c
Bottom col found: 9ce50845b
Bottom col found: 932d0f56b
Bottom col found: 235163358
Bottom col found: 665493ecc
Bottom col found: c8c4ef2e5
Bottom col found: 2127bb611
Bottom col found: b0c596a35
Leading col found: 5

Bottom col found: 89cebceab
Bottom col found: f171b61af
Bottom col found: 3a07a8939
Bottom col found: 129fe0263
Bottom col found: e5b2d137a
No bottom found
Leading col found: 79cc300c7
Leading col found: 20551fa5b
Bottom col found: 66ace2992
Bottom col found: f74e8f13d
Bottom col found: 5c6487af1
Bottom col found: 963a49cdc
No bottom found
Leading col found: 241f0f867
Leading col found: 2ec5b290f
Bottom col found: b8a716ebf
Bottom col found: d0f65188c
Bottom col found: adf357c9b
Bottom col found: cdbe394fb
No bottom found
Leading col found: d0f65188c
Leading col found: adf357c9b
Bottom col found: 17068424d
Bottom col found: 58d9f565a
Bottom col found: b77bc4dac
Bottom col found: 2a933bcb8
Bottom col found: dbb8e3055
Bottom col found: a8e607456
Bottom col found: a15b2f707
No bottom found
Leading col found: 58d9f565a
Leading col found: b77bc4dac
No bottom found
Leading col found: 324921c7b
Leading col found: 1702b5bf0
Bottom col found: f39074b55
Bottom col found: a8b721722
Bottom col fou

Bottom col found: 3f02111a8
Bottom col found: 9f8df01bd
Bottom col found: dc0118d47
Bottom col found: 8d468092c
Bottom col found: 7e293fbaf
Bottom col found: 83ce5d80d
Bottom col found: dc3b4460b
Bottom col found: 0a7ef2f9b
Bottom col found: 4c6a1accd
Bottom col found: 54b3d6afc
Bottom col found: d8aa77bcc
Bottom col found: 7b8ac394f
Bottom col found: ab7b12083
Bottom col found: 0024cd760
Bottom col found: 77854931e
Bottom col found: 1e991f09b
Bottom col found: 798a8e92e
Bottom col found: f967f827b
Bottom col found: 6f9d256a9
Bottom col found: 9401c82a9
Bottom col found: c0f2800fc
Bottom col found: 39d765b49
Bottom col found: ef6fab1bc
Bottom col found: 1a9b9e2fe
Bottom col found: e3a41c199
Bottom col found: 1529e64f6
Bottom col found: 2af3668d1
Leading col found: 03e726456
Leading col found: 7f17e0b2a
Bottom col found: 55f2b3d34
Bottom col found: f6c9661fc
Bottom col found: feed9d437
Bottom col found: 18321c252
Bottom col found: 22fbf6997
Bottom col found: d4815c074
Bottom col found: 

Bottom col found: 79e0c374a
Bottom col found: ad1466df8
Bottom col found: 007d71f12
Bottom col found: be448d5b9
Bottom col found: b0385cee8
Bottom col found: 8c8616b62
Bottom col found: b7f26c1f7
Bottom col found: 05f11f48f
Bottom col found: 2715b2d4a
Bottom col found: e1e8947d8
Bottom col found: 3d4a6baed
Bottom col found: 24bcc2f15
Bottom col found: 324aaa96f
Bottom col found: 34a2f580b
Bottom col found: b0310a768
Bottom col found: 0d866c3d7
Leading col found: 8de6fcbf1
Leading col found: a78f85d49
Bottom col found: 956d228b9
Bottom col found: 949ed0965
Bottom col found: a4511cb0b
Bottom col found: b64425521
No bottom found
Leading col found: 8618bc1fd
Leading col found: 64c6eb1cb
No bottom found
Leading col found: a3fb07bfd
Leading col found: 467bee277
No bottom found
Leading col found: deabe0f4c
Leading col found: ea5ed6ff7
Bottom col found: 2d7f256c6
Bottom col found: e98dd91da
Bottom col found: bce33962d
Bottom col found: cdaf3e576
Bottom col found: 78b2cf944
Bottom col found: b3

Leading col found: c768b2fbd
Leading col found: 44df9c106
Bottom col found: eae884486
Bottom col found: 468d2c3b6
Bottom col found: 57e185aad
Bottom col found: f72edfb37
Bottom col found: b6f5910aa
Bottom col found: 4a39584e5
Bottom col found: 951ef1246
Bottom col found: 76bfb8732
Bottom col found: 4a0e1a740
Bottom col found: fb5e1b2b7
Bottom col found: a1f9d1680
Bottom col found: d3b9b9a70
Bottom col found: 77697c671
Bottom col found: 0afb0ddcc
Bottom col found: 1189ee335
Bottom col found: bfbc53791
Bottom col found: 848b67fcc
Bottom col found: fc02e674d
Bottom col found: 4a8917f77
Bottom col found: 1401de8c2
Bottom col found: 2a6e64bb9
Bottom col found: cac875244
Bottom col found: 3e1100230
Bottom col found: 82f715995
Bottom col found: 59cafde1f
Bottom col found: 1d81e197a
Bottom col found: 3f8854df3
Bottom col found: 17b81a716
Bottom col found: 26cc05472
Bottom col found: 6786ea46d
Bottom col found: 1110cf9ea
Bottom col found: 621833d9b
Bottom col found: 5a798adc1
Bottom col found: 

Leading col found: 031490e77
Bottom col found: 4dcf81d65
Bottom col found: 61c1b7eb6
No bottom found
Leading col found: df838756c
No lead found
Bottom col found: 736513d36
Bottom col found: 0f07e3775
No bottom found
Leading col found: 408d86ce9
Leading col found: 79e55ef6c
No bottom found
Leading col found: 66891582e
Leading col found: 91de54e0a
Bottom col found: 2d6bd8275
Bottom col found: f3b4de254
No bottom found
Leading col found: d19110e37
Leading col found: 2be024de7
Bottom col found: 91de54e0a
Bottom col found: 66891582e
Bottom col found: 20ef8d615
No bottom found
Leading col found: 2b0fc604a
Leading col found: eb5a2cc20
Leading col found: 9df4daa99
Leading col found: ced6a7e91
No lead found
Leading col found: 34d3715d5
Leading col found: d0d340214
No lead found
Leading col found: c0004231c
Leading col found: 6cf7644e0
Leading col found: 48df886f9
Leading col found: 64e483341
Leading col found: 1d9078f84
No lead found
Leading col found: 58e2e02e6
Leading col found: f190486d6
No 

192

In [6]:

cols_2 = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11','73687e512',
                'aa164b93b','342e7eb03','cd24eae8a','8f3740670','2b2a10857','a00adf70e',
                '3a48a2cd2','a396ceeb9','9280f3d04','fec5eaf1a','5b943716b','22ed6dba3',
                '5547d6e11','e222309b0','5d3b81ef8','1184df5c2','2288333b4','f39074b55',
                'a8b721722','13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
                'b85fa8b27','2155f5e16']
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426',
        '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]
# x = extra_feats

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
for i, c in enumerate(extra_feats):
    if not i:
        x = pd.concat([train.loc[rows, ['target']+cols], train.loc[rows, c]], axis=1)
    else:
        x = pd.concat([x, train.loc[rows, c]], axis=1)
# display(train.loc[rows, extra_feats[25]])
display(x)

# Find leaks

In [7]:
# from: https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2
def _get_leak(df, cols, extra_feats=[[]], lag=0):
    f1 = cols[:-lag-2]
    f2 = cols[lag+2:]
    for ef in extra_feats:
        if len(ef) <= 2:
            continue
        f1 += ef[:-lag-2]
        f2 += ef[lag+2:]
    
    
    d1 = df[f1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[f2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [8]:
def compiled_leak_result(extra_cols, max_nlags=38, verbose=True):    
    train_leak = train
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        if verbose:
            print('Processing lag', i)
        train_leak[c] = _get_leak(train_leak, cols, extra_cols, i)
        
        leaky_cols.append(c)
#         train_leak = train.join(train_leak[leaky_cols+["compiled_leak", "nonzero_mean", "ID"]], on='ID', how='left')
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        if verbose:
            print("Leak values found in train", leaky_value_counts[-1])
            print(
                "% of correct leaks values in train ", 
                leaky_value_corrects[-1])
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = train_leak["nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        if verbose:
            print(
                'Score (filled with nonzero mean)', 
                scores[-1])
        gc.collect()
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

# Find helpful cols

In [9]:
good_cols = []
bad_cols = []
_, result = compiled_leak_result([[]], max_nlags=3, verbose=False)
ref = min(result['score'])
for i in range(len(extra_feats)):
    extra_cols = [extra_feats[i]]
    _, result = compiled_leak_result(extra_cols, max_nlags=3, verbose=False)
    if min(result['score']) < ref:
        print('Good col found, leader is ', extra_feats[i][0])
        good_cols.append(extra_feats[i])
    else:
        bad_cols.append(extra_feats[i])
print(len(good_cols), ' good columns found!')

Good col found, leader is  e222309b0
Good col found, leader is  cc507de6c
Good col found, leader is  b850c3e18
Good col found, leader is  226e2b8ac
Good col found, leader is  bae4f747c
Good col found, leader is  85fe78c6c
Good col found, leader is  2d065b147
Good col found, leader is  96b66294d
Good col found, leader is  1184df5c2
Good col found, leader is  e0bb9cf0b
Good col found, leader is  49ca7ff2e
Good col found, leader is  235b8beac
Good col found, leader is  9e7c6b515
Good col found, leader is  212efda42
Good col found, leader is  89cebceab
Good col found, leader is  8d8276242
Good col found, leader is  20551fa5b
Good col found, leader is  5d3b81ef8
Good col found, leader is  da22ed2b8
Good col found, leader is  86875d9b0
Good col found, leader is  678b3f377
Good col found, leader is  8618bc1fd
Good col found, leader is  64c6eb1cb
Good col found, leader is  6469d38ef
Good col found, leader is  a3e023f65
Good col found, leader is  ccc7609f4
Good col found, leader is  48b839509
G

In [10]:
# Apply fix to bad cols
def fix_bad_cols(df, bad_cols, verbose=True):
    _, result = compiled_leak_result([[]], max_nlags=38, verbose=verbose)
    ref = result['score']
    fixed_cols = []
    for i in range(len(bad_cols)):
        extra_cols = bad_cols[i]
        for j in range(1, len(extra_cols)-2):
            _, result = compiled_leak_result([extra_cols[:j+2]], max_nlags=j+1, verbose=verbose)
            if any([x > y for x,y in zip(result['score'], ref[:j+1])]):
                print('Bad cause found ', extra_cols[j+1])
#                 print('Attempting to fix it')
                fixed_cols.append(extra_cols[:j+1])
                break
    return fixed_cols
                
def fix_cols(extra_cols, bad_idx):
    pool = extra_cols[j+1:]
    pool.append(pool.pop(0))
    for k in range(len(pool)-1):
        print('Trying col ', pool[0])
        _, result = compiled_leak_result([extra_cols[:j+1]+[pool[0]]], max_nlags=j+1, verbose=verbose)
        if not any([x > y for x,y in zip(result['score'], ref[:j+1])]):
            print('Found new order ', pool[0])
        else:
            pool.append(pool.pop(0))

In [12]:
test_cols = [bad_cols[1]]
fixed_cols = fix_bad_cols(tmp, bad_cols, verbose=False)
[len(x) for x in fixed_cols]

Bad cause found  375c6080e
Bad cause found  861076e21
Bad cause found  1e8801477
Bad cause found  002d634dc
Bad cause found  cef9ab060
Bad cause found  d37030d36
Bad cause found  7da54106c
Bad cause found  39b3c553a
Bad cause found  1eec37deb
Bad cause found  597d78667
Bad cause found  f97d9431e
Bad cause found  7da54106c
Bad cause found  2ab5a56f5
Bad cause found  38ec5d3bb
Bad cause found  2b6f74f09
Bad cause found  7f55b577c
Bad cause found  fd99222ee
Bad cause found  b452ba57e
Bad cause found  d01cc5805
Bad cause found  208308a47
Bad cause found  7da54106c
Bad cause found  935ca66a9
Bad cause found  472cd130b
Bad cause found  c4972742d
Bad cause found  a728310c8
Bad cause found  3ecc09859
Bad cause found  da5c36eee
Bad cause found  7da54106c
Bad cause found  b64425521
Bad cause found  26ab20ff9
Bad cause found  ff2c9aa8f
Bad cause found  b6403de0b
Bad cause found  a4511cb0b
Bad cause found  03bfe48b2
Bad cause found  3aab2691c
Bad cause found  580f5ff06
Bad cause found  3aab2691c
B

[7,
 3,
 2,
 6,
 7,
 4,
 3,
 2,
 2,
 2,
 3,
 6,
 3,
 3,
 8,
 7,
 3,
 4,
 3,
 2,
 5,
 5,
 2,
 3,
 2,
 4,
 3,
 4,
 3,
 3,
 3,
 9,
 2,
 3,
 3,
 2,
 2,
 3,
 4,
 4,
 2,
 3,
 3,
 3,
 3,
 6,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 3,
 3,
 4,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 2,
 7,
 3,
 3,
 3,
 3,
 3,
 10,
 4,
 5,
 2,
 3,
 23,
 32,
 8,
 38,
 37,
 14,
 7,
 32,
 2,
 2,
 32,
 2,
 2]

In [13]:
# Append fixed cols to good cols
fixed_cols = [x for x in fixed_cols if len(x) >= 30]
print(len(fixed_cols), ' useful cols')
good_cols.extend(fixed_cols)
print('Total of ', len(good_cols), ' good cols')

22  useful cols
Total of  108  good cols


In [18]:
# Visualize bad_cols to understand what is happenning
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# len(bad_cols)
tmp.loc[:, bad_cols[1]]

,9a2b0a8be,856225035,f9db72cff,709573455,616be0c3e,19a67cb97,9d478c2ae,cf5b8da95,9c502dcd9,2f7b0f5b5,d50798d34,56da2db09,c612c5f8f,08c089775,7aaefdfd7,59cb69870,37c0a4deb,fb9a4b46d,b4eaa55ea,304633ac8,99f22b12d,65000b269,4bffaff52,4c536ffc0,93a445808,e8b513e29,a2616a980,97d5c39cf,71aae7896,62d0edc4f,c2acc5633,c8d5efceb,2e1287e41,2baea1172,af1e16c95,01c0495f8,b0c0f5dae,090f3c4f2,d834302b0
1757,34714000,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000,1500000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
3809,0,34714000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000,1500000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
511,0,0.0,34714000,1500000,1500000.0,1500000,1500000.0,1500000,1500000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
3798,0,0.0,0,34714000,1500000.0,1500000,1500000.0,1500000,1500000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
625,0,0.0,0,0,34714000.0,1500000,1500000.0,1500000,1500000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
3303,0,0.0,0,0,0.0,34714000,1500000.0,1500000,1500000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
4095,0,0.0,0,0,0.0,0,34714000.0,1500000,1500000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
1283,0,7238000.0,0,0,0.0,0,0.0,0,34714000,1500000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
4209,7276000,0.0,7238000,0,0.0,0,0.0,0,0,34714000,1500000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000
1696,0,7276000.0,0,7238000,0.0,0,0.0,0,0,0,34714000.0,1500000.0,1500000,1500000,1500000.0,1500000,1500000.0,1500000.0,1500000.0,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000,1500000.0,1500000,1500000,1500000,1500000,1500000,1500000,1500000


In [14]:
train_leak, result = compiled_leak_result(good_cols)
# train_leak, result = compiled_leak_result([[]])
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

Processing lag 0
Leak values found in train 1390
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.5138333391635181
Processing lag 1
Leak values found in train 2013
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.2795168110022879
Processing lag 2
Leak values found in train 2411
% of correct leaks values in train  0.9995852343425964
Score (filled with nonzero mean) 1.146765005376252
Processing lag 3
Leak values found in train 2671
% of correct leaks values in train  0.9996256083863722
Score (filled with nonzero mean) 1.046874179288105
Processing lag 4
Leak values found in train 2844
% of correct leaks values in train  0.999648382559775
Score (filled with nonzero mean) 0.9883074600053612
Processing lag 5
Leak values found in train 2997
% of correct leaks values in train  0.9996663329996663
Score (filled with nonzero mean) 0.9478608036692365
Processing lag 6
Leak values found in train 3119
% of correct leaks values in train  0.9996793844

In [12]:
result = pd.DataFrame.from_dict(result, orient='columns')
result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
leaky_correct,1.000000,1.000000,0.999585,0.999625,0.999648,0.999666,0.999679,0.999689,0.999697,0.999702,...,0.999477,0.999478,0.999478,0.999218,0.998959,0.998962,0.998704,0.998188,0.997672,0.997415
leaky_count,1388.000000,2010.000000,2407.000000,2667.000000,2840.000000,2993.000000,3115.000000,3217.000000,3303.000000,3357.000000,...,3822.000000,3828.000000,3834.000000,3838.000000,3844.000000,3853.000000,3859.000000,3864.000000,3866.000000,3869.000000
score,1.513833,1.279808,1.147272,1.047629,0.989107,0.948694,0.899494,0.851132,0.820565,0.800645,...,0.604919,0.602805,0.599573,0.597415,0.596461,0.594082,0.589793,0.588767,0.585366,0.585602


In [13]:
result.to_csv('train_leaky_stat.csv', index=False)

In [ ]:
train_leak.head()

In [19]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [16]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]
train_leak = rewrite_compiled_leak(train_leak, best_lag)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000d6aaf2,38000000.0,38000000.0,38000000.0,0.0,38000000.0,0.0,38000000.0,0.0,0.0,...,38000000.0,0.0,38000000.0,0.0,38000000.0,38000000.0,0.0,0.0,38000000.0,38000000.0
1,000fbd867,600000.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,600000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0
2,0027d6b71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0028cbf45,0.0,0.0,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0
4,002a68644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
train_res = train_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak.csv', index=False)

In [16]:
def compiled_leak_result_test(max_nlags, extra_cols):
    test_leak = test
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        test_leak[c] = _get_leak(test_leak, cols, extra_cols, i)
        
        leaky_cols.append(c)
#         test_leak = test.join(
#             test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
#             on="ID", how="left"
#         )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [17]:
test_leak, test_result = compiled_leak_result_test(38, good_cols)

Processing lag 0
Leak values found in test 2965
Processing lag 1
Leak values found in test 4227
Processing lag 2
Leak values found in test 4968
Processing lag 3
Leak values found in test 5534
Processing lag 4
Leak values found in test 5962
Processing lag 5
Leak values found in test 6258
Processing lag 6
Leak values found in test 6468
Processing lag 7
Leak values found in test 6622
Processing lag 8
Leak values found in test 6789
Processing lag 9
Leak values found in test 6926
Processing lag 10
Leak values found in test 7041
Processing lag 11
Leak values found in test 7136
Processing lag 12
Leak values found in test 7208
Processing lag 13
Leak values found in test 7283
Processing lag 14
Leak values found in test 7342
Processing lag 15
Leak values found in test 7404
Processing lag 16
Leak values found in test 7453
Processing lag 17
Leak values found in test 7492
Processing lag 18
Leak values found in test 7523
Processing lag 19
Leak values found in test 7571
Processing lag 20
Leak values 

In [21]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
leaky_count,2965,4227,4967,5533,5961,6257,6467,6622,6787,6922,...,7765,7779,7795,7802,7810,7823,7842,7876,7935,8120


In [22]:
test_result.to_csv('test_leaky_stat.csv', index=False)

In [20]:
test_leak = rewrite_compiled_leak(test_leak, best_lag)
test_leak[['ID']+['compiled_leak']].head()

,ID,compiled_leak
0,000137c73,0.0
1,00021489f,0.0
2,0004d7953,0.0
3,00056a333,0.0
4,00056d8eb,0.0


In [23]:
test_res = test_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak.csv', index=False)

In [ ]:
# test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = 0

In [21]:
#submission
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

baseline_sub_lag_36.csv saved


In [25]:
sub.head()

,ID,target
0,000137c73,0.0
1,00021489f,0.0
2,0004d7953,0.0
3,00056a333,0.0
4,00056d8eb,0.0


In [22]:
# Mix with other pred
sub_no_leak = pd.read_csv('sub_138.csv')
ensemble_sub = sub.copy()
ensemble_sub[sub['target'] == 0] = sub_no_leak[sub['target'] == 0]
ensemble_sub.to_csv(f"submission1.csv", index=False)
print(f"ensemble saved")
ensemble_sub.head()

ensemble saved


,ID,target
0,000137c73,1.751810e+06
1,00021489f,1.229325e+06
2,0004d7953,2.285429e+06
3,00056a333,5.295290e+06
4,00056d8eb,1.816481e+06
